In [1]:
import pandas as pd
from src.model.generate_stories import tfmclassifier

In [2]:
df = pd.read_csv('dataset/plot/tails.kwRAKE.csv', sep='\t')
df.head()

,[ID],[KEY/ABSTRACT],[KEYWORDS],[DISCOURSE (T/I/B/C)],[NUM_PARAGRAPHS],[PARAGRAPH],[PREVIOUS_PARAGRAPH]
0,plot-1_0,K,арысь поле[SEP]а в падчерицыно платье нарядила...,I,5,"У старика была дочь-красавица, жил он с нею ти...",NaN
1,plot-1_1,K,арысь поле[SEP]а в падчерицыно платье нарядила...,B,5,"Невзлюбила она падчерицу, пристала к старику","У старика была дочь-красавица, жил он с нею ти..."
2,plot-1_2,K,арысь поле[SEP]а в падчерицыно платье нарядила...,B,5,"- Прогони ее из дому, чтоб я ее и в глаза не в...","Невзлюбила она падчерицу, пристала к старику"
3,plot-1_3,K,арысь поле[SEP]а в падчерицыно платье нарядила...,B,5,Старик взял да и выдал свою дочку замуж,"- Прогони ее из дому, чтоб я ее и в глаза не в..."
4,plot-1_4,K,арысь поле[SEP]а в падчерицыно платье нарядила...,C,5,"Живет она с мужем да радуется, и родился у них...",Старик взял да и выдал свою дочку замуж


In [3]:
df['[DISCOURSE (T/I/B/C)]'].value_counts()

B    1026
I     360
C     360
Name: [DISCOURSE (T/I/B/C)], dtype: int64

In [4]:
df['[PARAGRAPH]'][0]

'У старика была дочь-красавица, жил он с нею тихо и мирно, пока не женился на другой бабе, а та баба была злая ведьма'

In [5]:
df['[PREVIOUS_PARAGRAPH]'].fillna('dummy', inplace=True)

In [6]:
prevpars = df['[PREVIOUS_PARAGRAPH]'].to_list()
print(len(prevpars))

1746


In [7]:
from transformers import AutoTokenizer, AutoModelWithLMHead
  
tokenizer = AutoTokenizer.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

model = AutoModelWithLMHead.from_pretrained("sberbank-ai/rugpt3small_based_on_gpt2")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
C:\Users\edbon\anaconda3\envs\tf\lib\site-packages\transformers\models\auto\modeling_auto.py:660: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  warnings.warn(


In [8]:
output = [(0,0,0)]
for i, par in enumerate(prevpars):
    output.append((i, par, tfmclassifier(par, model, tokenizer, gen_len=922))) 


RuntimeError: [enforce fail at ..\c10\core\CPUAllocator.cpp:79] data. DefaultCPUAllocator: not enough memory: you tried to allocate 328556544 bytes.